In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/masculinity-saturday-twitter-sentiment-analysis/twitter_sentiment_analysis_masculinity_saturday_data_annotated.csv', index_col=0)

In [3]:
df.head()

,tweet_created_at,text,Positive,Negative,Neutral,Compound,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,0.000,0.073,0.927,-0.2960,Negative
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,0.000,0.000,1.000,0.0000,Neutral
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,0.493,0.068,0.439,0.8271,Positive
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,0.000,0.180,0.820,-0.2960,Negative
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,0.110,0.000,0.890,0.6908,Positive


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 248738 entries, 0 to 248738
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   tweet_created_at  248738 non-null  object 
 1   text              248738 non-null  object 
 2   Positive          248738 non-null  float64
 3   Negative          248738 non-null  float64
 4   Neutral           248738 non-null  float64
 5   Compound          248738 non-null  float64
 6   Sentiment         248738 non-null  object 
dtypes: float64(4), object(3)
memory usage: 15.2+ MB


In [5]:
df.columns

Index(['tweet_created_at', 'text', 'Positive', 'Negative', 'Neutral',
       'Compound', 'Sentiment'],
      dtype='object')

In [6]:
df = df[['tweet_created_at', 'text', 'Sentiment']]

In [7]:
df.head()

,tweet_created_at,text,Sentiment
0,2022-12-30 22:37:46+00:00,Men who have refused to listen to/attend @amer...,Negative
1,2022-12-30 22:30:58+00:00,@OvOBrezzzy Take amerix next,Neutral
2,2022-12-30 22:18:00+00:00,@AyodeleYo11 @amerix Nah I’m good my masculini...,Positive
3,2022-12-30 20:42:54+00:00,@Amerix_DontSimp There is no need for dating i...,Negative
4,2022-12-30 20:32:23+00:00,@DyeAnna7 @amerix @kibe From what cos nothing ...,Positive


In [8]:
df['text'].head(20)

0     Men who have refused to listen to/attend @amer...
1                          @OvOBrezzzy Take amerix next
2     @AyodeleYo11 @amerix Nah I’m good my masculini...
3     @Amerix_DontSimp There is no need for dating i...
4     @DyeAnna7 @amerix @kibe From what cos nothing ...
5     @salim_the_don @MwendiaJnr I see you unblocked...
6     @german_kenyan @ML_Philosophy 🤣 yes they are a...
7     @PadreMusa Amerix once said the alcohol menace...
8     @DodoPicard You're in Eastafrica, he probably ...
9                      @HBK_ACTIVIST @amerix @kibe Heal
10                           Amerix student of the year
11    @sholla_saddam These are notes from @amerix's ...
12    @amerix I appreciate the intent of this, but I...
13    @Mynames_Jefff @Biancawamu2 @amerix Wouldn't b...
14    @sharbabababa @Biancawamu2 could be @amerix wife🤷
15    @DyeAnna7 @amerix @kibe Was raised by toxic co...
16    @butdarkacademia @purevibeezz good thing is An...
17    @amerix The system needs you sick and depr